In [1]:
import sys
import os
os.chdir("/playpen-nas-ssd/luchao/projects/SAM/")
import torch
from criteria import id_loss
import torchvision.transforms as transforms
from PIL import Image
import collections
import numpy as np
import torch.nn as nn

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
id_identifier = id_loss.IDLoss().to('cuda').eval()
img_transforms = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])

Loading ResNet ArcFace


/playpen-nas-ssd/luchao/projects/SAM/criteria/id_loss.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.facenet.load_state_dict(torch.load(model_paths['ir_se50']))


In [3]:
def iterate_over_img_dir(img_dir):
    IMG_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if any(file.endswith(extension) for extension in IMG_EXTENSIONS):
                yield os.path.join(root, file)

In [4]:
def get_reference_img_features(reference_img_dir):
    feats_dict = collections.defaultdict(list)
    for img_name in os.listdir(reference_img_dir):
        img_path = os.path.join(reference_img_dir, img_name)
        img = Image.open(img_path)
        img = img_transforms(img)
        img = img.unsqueeze(0).to('cuda')
        feat = id_identifier.extract_feats(img)
        age = int(img_name.split('_')[0])
        assert 0 <= age <= 100
        feats_dict[age].append(feat)
    return feats_dict

def calc_id_similarities(img_dir, feats_dict):
    results_id_similarity = collections.defaultdict(list)
    # pretrained = any([keyword in single_img_dir for keyword in ['pretrained', 'AgeTransGAN', 'CUSP', 'FADING']])
    # if pretrained:
    #     print(f"Getting results for baseline (pretrained, AgeTransGAN, CUSP, FADING)")
    #     pass
    # else:
    #     # print(f'Getting results for personalized model')
    #     train_min_age = int(single_img_dir.split('/')[-2].split('_')[1])
    #     train_max_age = int(single_img_dir.split('/')[-2].split('_')[2])
    #     # print(f'Train min age: {train_min_age}')
    #     # print(f'Train max age: {train_max_age}')
    res = []
    for root, dirs, files in os.walk(img_dir):
        for img_name in files:
            img_path = os.path.join(root, img_name)

            # ----------------------------------- todo ----------------------------------- #
            # if img_name.count('_') == 1:
            #     # print(f'Skip input image: {img_name}')
            #     continue
            if any([keyword in img_name for keyword in ['inversion']]):
                # print(f'Skip inversion image: {img_name}')
                continue
            
            target_age = int(img_name.split('.')[0].split('_')[-1])
            # print(f'Processing {img_name}')
            # print(f'Target age: {target_age}')
            img = Image.open(img_path)
            img = img_transforms(img)
            img = img.unsqueeze(0).to('cuda')
            assert 0 <= target_age <= 100
            predict_img_id_feats = id_identifier.extract_feats(img)
            reference_feats = [feat[0] for k in feats_dict.keys() if abs(k - target_age) <= 3 for feat in feats_dict[k]]
            if len(reference_feats) == 0:
                # print(f'No reference feature for {target_age}')
                continue
            reference_feats = torch.stack(reference_feats)
            # find the most similar reference feature to the predicted feature (cosine similarity)
            id_similarity = nn.functional.cosine_similarity(predict_img_id_feats, reference_feats).max().item()
            results_id_similarity[target_age].append(id_similarity)

            results_id_similarity_mean = {k: np.mean(v) for k, v in results_id_similarity.items()}
            results = np.mean([v for k, v in results_id_similarity_mean.items()])

            res.append(results)
    # ! change here
    results_interpolation = np.mean([v for k, v in results_id_similarity_mean.items() if 30<=k<=70])
    # results_interpolation = np.mean([v for k, v in results_id_similarity_mean.items() if 50<=k<=70])
    results_extrapolation = np.mean([v for k, v in results_id_similarity_mean.items() if k<=70])

    # results_interpolation = np.mean([v for k, v in results_id_similarity_mean.items() if 40<=k<=60])
    # results_extrapolation = np.mean([v for k, v in results_id_similarity_mean.items() if k>=40])
    # return results_interpolation, results_extrapolation
    return np.mean(res)

In [5]:
ours = []
# mytm = []
fading = []
flowedit = []
rfsolver = []
rfinversion = []
fireflow = []

In [8]:
celebs = ['al', 'charles', 'chow', 'diego', 'elaine', 'elizabeth', 'jackie', 'jennifer', 'nicole', 'oprah', 'robert', 'thatcher']

for celeb in celebs:
    reference_img_dir = f'/playpen-nas-ssd/luchao/projects/SAM/age_dataset/{celeb}/mystyle'
    feats_dict = get_reference_img_features(reference_img_dir)

    exp_parent_dir = f'/playpen-nas-ssd/gongbang/comparisons/fireflow/{celeb}/'

    for exp in sorted(os.listdir(exp_parent_dir)):
        exp_dir = os.path.join(exp_parent_dir, exp)

        results_id_similarity_mean = calc_id_similarities(exp_dir, feats_dict)
        fireflow.append(results_id_similarity_mean)
        print(f'Experiment: {exp_dir}')
        print(f'ID similarity: {results_id_similarity_mean}')
        print('------------------------------------')


Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/al/alcoholism
ID similarity: 0.5512512952690554
------------------------------------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/al/goodskin
ID similarity: 0.5692490594586498
------------------------------------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/al/hairloss
ID similarity: 0.5710833497649506
------------------------------------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/al/poorskin
ID similarity: 0.5584516798284378
------------------------------------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/al/weight
ID similarity: 0.5563941529595737
------------------------------------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/charles/alcoholism
ID similarity: 0.6622761257764553
------------------------------------
Experiment: /playpen-nas-ssd/gongbang/comparisons/fireflow/charles/goodskin
ID similarity: 0.6496707915474828
-------------------------------

In [9]:
np.mean(fireflow)

0.5220906539258287